### Step 2: merge all .csv files into a one-year dataframe

loop through the lob_caps directory, forming one time-sorted dataframe, with all CAPS files. These files captured sample bid and ask capitalization, and respective bid and ask volumes.

In [36]:
!pip3 install matplotlib
!pip3 install altair

In [37]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [38]:
!mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

grep: ./lob_caps: Is a directory
mv: missing destination file operand after 'backup_match/'
Try 'mv --help' for more information.


In [39]:
#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  298660
start:  1660221600292.0  end:  1693078943553.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


In [40]:
# def getSKEWByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
#                                 # print("for type, ", type)
#     ret = []
#     for root, dirs, files in os.walk("./lob_caps/"):
#         for filename in files:
#             if type in filename:
# #                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
#                 ret.append(filename)
#     return ret

# csvFileList = getSKEWByDateAndType("MEANSHIFT") #iterate this array to dip into each csv, later on
# li = []                         #form the endFrame / global data frame around this array
# for filename in csvFileList:
#     csv = "lob_caps/" + filename
#     # print(csv)
#     df = pd.read_csv(csv, index_col=None, header=0)
#     li.append(df)

# skewFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
# skewFrame.sort_values(by=['timeStamp'], ascending=True)   #sorted by time into one time series
# skewFrame.rename(columns={'timeStamp': 'time'}, inplace=True)
# print("for new df: ", skewFrame.shape[0])
# start = skewFrame["time"].min()
# end = skewFrame["time"].max()
# print("start: ", start, " end: ", end)
# print(skewFrame.columns)

In [41]:
# Merging those two data frames Will not take place based on a Shared key of time
# You'll need to look up the SKU value for every row based on approximation
# merged_df = pd.merge(capsFrame, skewFrame, on='time')

In [42]:
# skewFrame.head(10)

In [43]:
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'mean'] = skewFrame['mean'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'skew'] = skewFrame['skew'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().lt(pd.Timedelta(minutes=2)).idxmax(), 'mean'] = skewFrame['mean'].values[0]
# 
# Could not get this operation to work try it again using a range of values once the precursor and surge are defined


## schema for capitalization data

loads the csv files, as acquired from coinbase

In [44]:
capsFrame.head(2)

,bc,ac,tbv,tav,time,mp,minBid
0,2828853.84,10260926.86,541682.81,221178.78,1.672934e+12,11.76,11.71
1,2826677.43,10256711.31,541497.46,220827.12,1.672934e+12,11.76,11.71


## Discover precursor and surge episodes

the goal of the data prep is to discover periods of continuous, positive momentum. These are **surges**. 

The periods preceding surges are, for the sake of the experiment, **precursors**. They are detected as periods of discontinuous positive momentum, or negative momentum. 

A ten-row window is used to calculate positive or negative momentum. A percent **change** is calculated for the ten row subsample.

In [45]:
# Load your time series data into a pandas dataframe
caps_df = capsFrame   
lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

298660 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change'], dtype='object')


###  establish benchmarks for percent change

the mean of change represents the average rate of change between LOB samples. This is used to determine whether the change between rows is significant or not. 

In [46]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

0.0005284

## define precursors from surges

use the threshold, mean change as tool to separate precursor from surges, where surges represent periods of positive momentum above threshold.

This step defines the data schema for the remainder of the process, where key statistics are defined for precursors and surges.

In [48]:
# identify units of 10 rows where the percent change is greater or less than the threshold
threshold = meanChange
surges = []
precursors = []

for i in range(0,len(caps_df),10):

    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                            'type':'precursor',
                            'p_buyCap':caps_df.iloc[i]['bc'], 
                            'p_askCap':caps_df.iloc[i]['ac'],
                            'p_totalBidVol':caps_df.iloc[i]['tbv'],
                            'p_totalAskVol':caps_df.iloc[i]['tav']
                            })  

In [49]:
# for item in surges:
#     print(item)

In [50]:
# for item in precursors:
#     print(item)

## merge precursors and surges into time series

a dataframe of sequences, **sequence_df** is created by concatenating both buckets, and sorting by time. This will create a time series of surge and precursor periods, as defined by: 

* 10 window percent change values
* contiguity: these precursor and surges are next to each other and thus have a length or duration of momentum.

In [51]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)

In [52]:
sequence_df.index

Int64Index([ 9696, 18333, 18334, 18335, 18336, 18337, 18338,  9697, 18339,
             9698,
            ...
            10476,  5635, 10477, 10478, 10479, 10480, 10481,  5636,  5637,
             5638],
           dtype='int64', length=29866)

### view the aligned, continuous time series of precursors and surges

view the final abstraction: sets of precursor periods, next to surges, in a linear time series. Precursors effectively precede surges on a linear time series.

In [53]:
# for index, row in sequence_df.iterrows():
#     print(row['surge'], row['precursor'])
sequence_df['type'].head(40)

9696         surge
18333    precursor
18334    precursor
18335    precursor
18336    precursor
18337    precursor
18338    precursor
9697         surge
18339    precursor
9698         surge
9699         surge
18340    precursor
18341    precursor
18342    precursor
9700         surge
9701         surge
18343    precursor
18344    precursor
9702         surge
18345    precursor
18346    precursor
18347    precursor
18348    precursor
18349    precursor
18350    precursor
18351    precursor
9703         surge
9704         surge
18352    precursor
18353    precursor
9705         surge
9706         surge
9707         surge
9708         surge
18354    precursor
18355    precursor
18356    precursor
18357    precursor
9709         surge
18358    precursor
Name: type, dtype: object

In [54]:
# sequence_df.head(45)

## visualize proof of algorithmic accuracy

this chart will plot the price time series, with an area of precursor and surge, as proof of our algorithmic accuracy.

In [55]:

# line = alt.Chart(sequence_df).mark_line(color='green').encode(
#     x='time',
#     y='s_MP'
# )

# s_bar = alt.Chart(sequence_df).mark_bar().encode(
#     x='time',
#     y='s_MP',
#     color='type:N'
# )

# p_bar = alt.Chart(sequence_df).mark_bar().encode(
#     x='time',
#     y='p_MP',
#     color='type:N'
# )

# chart = (line + s_bar + p_bar).properties(width=600, height=500)
# chart.title = 'Sequential order of precursor and surges for April 7th 2023'
# subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
# chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
# chart

In [56]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol'],
      dtype='object')

### Perform information gain on grouped precursors and surges

define the **sum change**, or total change per continuous episode (precursor or surge). 

define the **length** of each episode. 

define the height of the surge, how high did the continuous positive momentum reach?

define the size (area) of the surge, as a triangular area (height times length), as **surge_area**

Create one line to describe a precursor or search and it's related order book statistics

In [57]:

sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')
# sequence_df['identifier'] = sequence_df.groupby(['type', 'group'])['time'].transform('min') #prep the label early, if surge?
print(sequence_df.shape[0])
sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')
print(sequence_df.shape[0])

sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)
sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']
sequence_df.columns
# sequence_df['area'] = sequence_df.groupby(['type', 'group'])['sum_change'].multiply(sequence_df['length'])
# sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] =  sequence_df['length'] * sequence_df['sum_change']


# sequence_df['sum'] = sequence_df.groupby(['surge', 'group'])['change'].transform('sum')

# sequence_df['end_time'] = sequence_df.groupby(['surge', 'group'])['time'].transform('max')
# sequence_df['type'] = sequence_df['surge']

# sequence_df['buyCapSum'] = sequence_df.groupby(['surge', 'group'])['buyCap'].transform('avg')
# sequence_df['askCapSum'] = sequence_df.groupby(['surge', 'group'])['askCap'].transform('avg')

# calculate the area for the surge

# sequence_df = sequence_df.drop('next_value', axis=1)
# sequence_df.loc[sequence_df['bucket'] == 'surge', 'surge_length'] =  sequence_df['length']
# sequence_df.drop('length', axis=1, inplace=True)
# df = df.loc[:,~df.columns.duplicated()]

#unique_df = sequence_df.groupby('identifier').first().reset_index()

# unique_df.loc[unique_df['surge'] == '1', 'surge_length'] = unique_df['length']
# unique_df.loc[unique_df['surge'] == '0', 'length'] = 0

29866
29866


Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol', 'group', 'length', 'sum_change',
       'area', 'surge_area'],
      dtype='object')

In [58]:
sequence_df.head(20)

,time,s_MP,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,group,length,sum_change,area,surge_area
9696,1.660222e+12,30.00,0.505364,surge,NaN,NaN,NaN,NaN,NaN,1,1,5.053641e-01,0.505364,0.505364
18333,1.660222e+12,NaN,-0.000533,precursor,29.98,7306552.60,1.229787e+09,715246.91,314376.38,2,6,-5.009518e-03,-0.030057,NaN
18334,1.660222e+12,NaN,-0.001067,precursor,29.98,7339209.15,1.229766e+09,716217.12,313647.07,2,6,-5.009518e-03,-0.030057,NaN
18335,1.660222e+12,NaN,-0.001035,precursor,29.93,7317434.26,1.229763e+09,715583.54,313570.01,2,6,-5.009518e-03,-0.030057,NaN
18336,1.660222e+12,NaN,-0.001136,precursor,29.88,7312691.33,1.229773e+09,715470.64,313927.53,2,6,-5.009518e-03,-0.030057,NaN
18337,1.660222e+12,NaN,-0.000067,precursor,29.88,7313881.63,1.229775e+09,715515.08,314010.16,2,6,-5.009518e-03,-0.030057,NaN
18338,1.660222e+12,NaN,-0.001171,precursor,29.85,7310640.44,1.229804e+09,715383.42,315013.70,2,6,-5.009518e-03,-0.030057,NaN
9697,1.660222e+12,29.86,0.000670,surge,NaN,NaN,NaN,NaN,NaN,3,1,6.702414e-04,0.000670,0.000670
18339,1.660222e+12,NaN,0.000135,precursor,29.89,7293630.54,1.229796e+09,714871.56,314771.19,4,1,1.345640e-04,0.000135,NaN
9698,1.660222e+12,29.88,0.001273,surge,NaN,NaN,NaN,NaN,NaN,5,2,1.807711e-03,0.003615,0.003615


## Critical group by unique identifier

In [59]:
unique_df = sequence_df.groupby('group').first().reset_index()
print(unique_df)

       group          time   s_MP    change       type   p_MP    p_buyCap  \
0          1  1.660222e+12  30.00  0.505364      surge    NaN         NaN   
1          2  1.660222e+12    NaN -0.000533  precursor  29.98  7306552.60   
2          3  1.660222e+12  29.86  0.000670      surge    NaN         NaN   
3          4  1.660222e+12    NaN  0.000135  precursor  29.89  7293630.54   
4          5  1.660222e+12  29.88  0.001273      surge    NaN         NaN   
...      ...           ...    ...       ...        ...    ...         ...   
11268  11269  1.693073e+12  10.03  0.000898      surge    NaN         NaN   
11269  11270  1.693073e+12    NaN  0.000000  precursor  10.03  2176012.73   
11270  11271  1.693076e+12  10.02  0.000699      surge    NaN         NaN   
11271  11272  1.693076e+12    NaN  0.000000  precursor  10.02  2190863.39   
11272  11273  1.693078e+12  10.02  0.001298      surge    NaN         NaN   

           p_askCap  p_totalBidVol  p_totalAskVol  length  sum_change  \
0 

### Merge even and odd Rows to form the final sequences

Even rows contain surge, and odd rows contain precursors. **When you merge them, you form a sequence of precursor, and surge.**

Each row will contain a continuous **precursor->surge** sequence.

In [60]:
even_df = unique_df.iloc[::2].reset_index(drop=True)
odd_df = unique_df.iloc[1::2].reset_index(drop=True)

merged_df = pd.concat([even_df, odd_df], axis=1)

print(merged_df)

      group          time   s_MP    change   type  p_MP  p_buyCap  p_askCap  \
0         1  1.660222e+12  30.00  0.505364  surge   NaN       NaN       NaN   
1         3  1.660222e+12  29.86  0.000670  surge   NaN       NaN       NaN   
2         5  1.660222e+12  29.88  0.001273  surge   NaN       NaN       NaN   
3         7  1.660222e+12  29.80  0.000873  surge   NaN       NaN       NaN   
4         9  1.660223e+12  29.90  0.001305  surge   NaN       NaN       NaN   
...     ...           ...    ...       ...    ...   ...       ...       ...   
5632  11265  1.693072e+12  10.02  0.109437  surge   NaN       NaN       NaN   
5633  11267  1.693073e+12  10.02  0.000599  surge   NaN       NaN       NaN   
5634  11269  1.693073e+12  10.03  0.000898  surge   NaN       NaN       NaN   
5635  11271  1.693076e+12  10.02  0.000699  surge   NaN       NaN       NaN   
5636  11273  1.693078e+12  10.02  0.001298  surge   NaN       NaN       NaN   

      p_totalBidVol  p_totalAskVol  ...       type 

In [61]:
nan_cols = merged_df.dropna(axis=1, how='all')
nan_cols.head()

,group,time,s_MP,change,type,length,sum_change,area,surge_area,group,...,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,length,sum_change,area
0,1,1.660222e+12,30.00,0.505364,surge,1,0.505364,0.505364,0.505364,2.0,...,-0.000533,precursor,29.98,7306552.60,1.229787e+09,715246.91,314376.38,6.0,-5.009518e-03,-0.030057
1,3,1.660222e+12,29.86,0.000670,surge,1,0.000670,0.000670,0.000670,4.0,...,0.000135,precursor,29.89,7293630.54,1.229796e+09,714871.56,314771.19,1.0,1.345640e-04,0.000135
2,5,1.660222e+12,29.88,0.001273,surge,2,0.001808,0.003615,0.003615,6.0,...,-0.000233,precursor,29.94,7348772.25,1.229748e+09,716600.57,313351.78,3.0,-3.410602e-03,-0.010232
3,7,1.660222e+12,29.80,0.000873,surge,2,0.002114,0.004229,0.004229,8.0,...,-0.000502,precursor,29.87,7323907.02,1.229738e+09,715906.34,313373.57,2.0,8.855895e-07,0.000002
4,9,1.660223e+12,29.90,0.001305,surge,1,0.001305,0.001305,0.001305,10.0,...,-0.000768,precursor,29.95,7312223.63,1.229791e+09,715186.92,314804.24,7.0,-1.377036e-02,-0.096393


In [62]:
nan_cols.columns

Index(['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
       'area', 'surge_area', 'group', 'time', 'change', 'type', 'p_MP',
       'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'length',
       'sum_change', 'area'],
      dtype='object')

### Write to CSV: step one, pipeline
Label to use is surge_area

In [65]:
# nan_cols = nan_cols.rename(columns={'group': 'group_1', 'time': 'time_1', 'change': 'change_1', 'type': 'type_1', 'length': 'length_1', 'sum_change': 'sum_change_1', 'area': 'area_1'})
# writeable_df = nan_cols['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change','area', \
#                         'surge_area', 'group', 'time', 'change', 'type', 'p_MP',
#                        'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'length','sum_change', 'area']

# writeable_df.columns = ['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
#        's_area', 'surge_area', 'p_group', 'p_time', 'p_change', 'p_type', 'p_MP',
#        'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'p_length',
#        'p_sum_change', 'p_area']
nan_cols.to_csv('pipeline1.csv', index=False)
# df.to_csv('filename.csv', index=False)
# writeable_df.to_csv('pipeline1.csv', index=False)


In [ ]:
'''The repeating elements in the list are:
- group
- time
- change
- type
- length
- sum_change
- area '''